In [1]:
import xarray
import openeo

import openeo
from openeo.processes import exp, array_element, log 
from openeo.extra.spectral_indices.spectral_indices import append_index
from openeo.udf.debug import inspect
from datetime import datetime
from dateutil.relativedelta import *

In [2]:
connection = openeo.connect(
    url="openeo-dev.vito.be"
    # url="openeo.vito.be"
).authenticate_oidc()

Authenticated using refresh token.


# S2 Collection

In [3]:
start_date      = '2022-01-03'
end_date        = '2022-01-08'
spatial_extent  = {'west': -74.06810760, 'east': -73.90597343, 'south': 4.689864510, 'north': 4.724080996, 'crs': 'epsg:4326'}

# Selecting B03 was needed to make sure that the angles bands have 10 m resolution
s2_cube = connection.load_collection(
    'SENTINEL2_L2A_SENTINELHUB',
    spatial_extent = spatial_extent,
    temporal_extent = [start_date, end_date],
    bands = ['B03', 'sunAzimuthAngles', 'sunZenithAngles'] 
)

# DEM data (COPERNICUS_30)

In [4]:
dem_cube = connection.load_collection(
    "COPERNICUS_30",
    spatial_extent = spatial_extent,
    temporal_extent=["2010-01-01", "2030-12-31"],
)

dem_cube = dem_cube.max_time()
dem_cube = dem_cube.resample_cube_spatial(s2_cube)
merged_cube = s2_cube.merge_cubes(dem_cube)

# Hillshade process with UDF

In [5]:
udf_code = """

from openeo.udf import XarrayDataCube
from openeo.udf.debug import inspect
import numpy as np
from hillshade.hillshade import hillshade


def rasterize(azimuth, resolution=None):
    # Convert the azimuth into its components on the XY-plane. Depending on the value of the
    # azimuth either the x or the y component of the resulting vector is scaled to 1, so that
    # it can be used conveniently to walk a grid.

    azimuth = np.deg2rad(azimuth)
    xdir, ydir = np.sin(azimuth), np.cos(azimuth)

    if resolution is not None:
        xdir = xdir * resolution[0]
        ydir = ydir * resolution[1]

    slope = ydir / xdir
    if slope < 1. and slope > -1.:
        xdir = 1.
        ydir = slope
    else:
        xdir = 1. / slope
        ydir = 1.
    return xdir, ydir


def _run_shader(sun_zenith, sun_azimuth, elevation_model, resolution_x, resolution_y):

    azimuth = np.nanmean(sun_azimuth.astype(np.float32))
    zenith = np.nanmean(sun_zenith.astype(np.float32))
    resolution = (float(resolution_x), float(resolution_y))
    ray_xdir, ray_ydir = rasterize(azimuth, resolution)

    # Assume chunking is already done by Dask
    ystart = 0
    yend = elevation_model.shape[0]

    # Make sure inputs have the right data type
    zenith = float(zenith)
    ray = (float(ray_xdir), float(ray_ydir))
    shadow = hillshade(elevation_model.astype(np.float32),
                       resolution,
                       zenith,
                       ray,
                       ystart,
                       yend)
    shadow = shadow.reshape(elevation_model.shape)
    shadow[np.isnan(sun_azimuth)] = 255
    return shadow


def apply_datacube(cube: XarrayDataCube, context: dict) -> XarrayDataCube:
    in_xarray = cube.get_array()
    sun_zenith = in_xarray.sel({"bands": "sunZenithAngles"}).values.astype(np.float32)
    sun_azimuth = in_xarray.sel({"bands": "sunAzimuthAngles"}).values.astype(np.float32)
    elevation_model = in_xarray.sel({"bands": "DEM"}).values.astype(np.float32)
    res_y = in_xarray.coords["y"][int(len(in_xarray.coords["y"])/2)+1] - in_xarray.coords["y"][int(len(in_xarray.coords["y"])/2)]
    res_x = in_xarray.coords["x"][int(len(in_xarray.coords["x"])/2)+1] - in_xarray.coords["x"][int(len(in_xarray.coords["x"])/2)]

    shadow = _run_shader(sun_zenith, sun_azimuth, elevation_model, res_x, res_y)
    cube.get_array().values[0] = shadow
    return cube

"""

In [ ]:
process = openeo.UDF(code=udf_code, runtime="Python", data={"from_parameter": "x"})

# Shadow mask replaces the first band (B03) in the merged cube
hillshaded = merged_cube.apply(process=process)

my_job  = hillshaded.send_job(title="hillshaded")
results = my_job.start_and_wait().get_results()
results.download_files("hilshaded")

hillshaded = hillshaded.rename_labels("bands", ["hillshade_mask", "sunAzimuthAngles", "sunZenithAngles", "DEM"])


C:\Users\rmgu\AppData\Local\Temp\ipykernel_36200\3607831980.py:6: DeprecationWarning: Call to deprecated method `send_job`, use `create_job` instead.
  my_job  = hillshaded.send_job(title="hillshaded")


0:00:00 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': send 'start'
0:00:31 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:00:36 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:00:43 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:00:50 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:01:01 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:01:13 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:01:28 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:01:47 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:02:11 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:02:41 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:03:19 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:04:05 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': queued (progress N/A)
0:05:04 Job 'j-592ce3aad6df4c4ba512862bd17b6c0f': running (progress N/A)


In [72]:
my_job.logs()

[{'id': 'openeo-yarn-index-1m-2022.10.18-000013/IOY4N4QBMcjMPc-fXQ_t',
  'time': '2022-11-02T07:22:30.620Z',
  'level': 'info',
  'message': "Submitting job: ['/opt/venv/lib64/python3.8/site-packages/openeogeotrellis/deploy/submit_batch_job_spark3.sh', 'openEO batch_hillshaded_j-3bf0e39847ee4745803808be3996d20b_user 9b8a05ab83d3da5df69769fd81ba1e5a010e257d100ae51910454badf52bd5d5@egi.eu', '/data/projects/OpenEO/j-3bf0e39847ee4745803808be3996d20b_7n9vhfzr.in', '/data/projects/OpenEO/j-3bf0e39847ee4745803808be3996d20b', 'out', 'log', 'job_metadata.json', 'openeo@VGT.VITO.BE', '/opt/openeo.keytab', 'openeo', '1.1.0', '8G', '2G', '3G', '5', '2', '2G', 'default', 'false', '[]', 'custom_processes.py', '100', '9b8a05ab83d3da5df69769fd81ba1e5a010e257d100ae51910454badf52bd5d5@egi.eu', 'j-3bf0e39847ee4745803808be3996d20b', '0.0', '1', 'default', '/data/projects/OpenEO/j-3bf0e39847ee4745803808be3996d20b_jfwzgqe2.properties']"},
 {'id': 'openeo-yarn-index-1m-2022.10.18-000013/HuY4N4QBMcjMPc-fXQ_t',
  'time': '2022-11-02T07:22:30.620Z',
  'level': 'info',
  'message': 'deemed collection SENTINEL2_L2A_SENTINELHUB AOI (68062397.83501363 m²) too small for batch processing (threshold 2500000000 m²)'},
 {'id': 'openeo-yarn-index-1m-2022.10.18-000013/kfQ4N4QBVWXUH_mWXe_v',
  'time': '2022-11-02T07:22:30.620Z',
  'level': 'info',
  'message': "Dry run extracted these source constraints: [(('load_collection', ('SENTINEL2_L2A_SENTINELHUB', ())), {'temporal_extent': ('2022-01-03', '2022-01-08'), 'spatial_extent': {'west': -74.0681076, 'south': 4.68986451, 'east': -73.90597343, 'north': 4.724080996, 'crs': 'epsg:4326'}, 'bands': ['B05', 'sunAzimuthAngles', 'sunZenithAngles']}), (('load_collection', ('COPERNICUS_30', ())), {'temporal_extent': ('2010-01-01', '2030-12-31'), 'spatial_extent': {'west': -74.0681076, 'south': 4.68986451, 'east': -73.90597343, 'north': 4.724080996, 'crs': 'epsg:4326'}, 'process_type': [<ProcessType.GLOBAL_TIME: 4>]}), (('load_collection', ('SENTINEL2_L2A_SENTINELHUB', ())), {'temporal_extent': ('2022-01-03', '2022-01-08'), 'spatial_extent': {'west': -74.0681076, 'south': 4.68986451, 'east': -73.90597343, 'north': 4.724080996, 'crs': 'epsg:4326'}, 'bands': ['B05', 'sunAzimuthAngles', 'sunZenithAngles']})]"},
 {'id': 'openeo-yarn-index-1m-2022.10.18-000013/lPQ4N4QBVWXUH_mWXe_v',
  'time': '2022-11-02T07:22:30.620Z',
  'level': 'info',
  'message': 'deemed collection SENTINEL2_L2A_SENTINELHUB AOI (68062397.83501363 m²) too small for batch processing (threshold 2500000000 m²)'},
 {'id': 'openeo-yarn-index-1m-2022.10.18-000013/kuY4N4QBMcjMPc-fsBsh',
  'time': '2022-11-02T07:22:51.622Z',
  'level': 'info',
  'message': '  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current\n                                 Dload  Upload   Total   Spent    Left  Speed\n\n  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\n100  1429  100  1300  100   129  34210   3394 --:--:-- --:--:-- --:--:-- 37605\n\n100  1726  100  1597  100   129  16635   1343 --:--:-- --:--:-- --:--:-- 17978\n{"result": {"count": 3, "truncated": false, "messages": [{"type": "warning", "message": "API Version number was not sent, forward compatibility not guaranteed. Assuming server\'s API version, 2.231", "code": 13001, "data": {"server_version": "2.231"}, "name": "VersionMissing"}], "result": [{"dn": "uid=openeo,cn=users,cn=accounts,dc=vgt,dc=vito,dc=be", "uid": ["openeo"], "krbcanonicalname": ["openeo@VGT.VITO.BE"], "loginshell": ["/bin/bash"], "uidnumber": ["631703554"], "gidnumber": ["631703554"], "sn": ["TERRA-1378"], "homedirectory": ["/home/openeo"], "mail": ["openeo@vgt.vito.be"], "krbprincipalname": ["openeo@VGT.VITO.BE"], "givenname": ["openeo user"], "nsaccountlock": false}, {"dn": "uid=openeo-dev,cn=users,cn=accounts,dc=vgt,dc=vito,dc=be", "uid": ["openeo-dev"], "krbcanonicalname": ["openeo-dev@VGT.VITO.BE"], "loginshell": ["/bin/bash"], "uidnumber": ["631704057"], "gidnumber": ["631704057"], "sn": ["dev"], "hom